In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import preprocess as prep

In [ ]:
def corner_point(position):
    x1, y1, x2, y2 = position
    return min(x1, x2), min(y1, y2)

In [ ]:
def data_refresh(data, x_final, y_final, window_shape_value):
    data_refresh = data[y_final:y_final+window_shape_value, x_final:x_final+window_shape_value]
    return data_refresh

In [ ]:
# Save the data and asteroid position into a npz file
def save_data_negative(name, index, data, position, data_path):
    dataPath = '{}\{}'.format(data_path, name)
    np.savez(dataPath, matrix1=data)
    print('Successfully save data for {}: {}'.format(index, name))

In [ ]:
def generate_negative_data(father_dir, pic_path, data_path, start_index, end_index, window_shape_value=1000):
    count = 0
    
    for index in range(start_index, end_index+1):
        path, name = prep.path_of_data(father_dir, index)
        print('Start preprocessing {}: {}'.format(index, name))
        data, position = prep.load_saved_data(path)
        x_corner, y_corner = corner_point(position)
        if x_corner > window_shape_value and y_corner > window_shape_value:
            for i in range(2):
                x_final = random.randint(0, x_corner - window_shape_value)
                y_final = random.randint(0, y_corner - window_shape_value)
                data_new = data_refresh(x_final, y_final, window_shape_value)
                
            